# Perception Algorithm: Spam Filter
Seung Heon (Brian) Shin - Shs522

Steps:

### 1 - <b>Dividing the data set </b>


a) Training Set (set of 4000)

b) Validation Set (set of 1000)

### 2 - <b>Transforming Data into Feature Vectors</b>

- removing words that occur is less than 30 emails

### 3 - <b>Building the perceptron algorithm</b>

a) perceptron train(data): trains the perceptron classifier using the data provided

b) perceptron test(w, data): returns test error

- Ignore all words that appear in fewer than X = 30 e-mails

### 4 - Training the perception classifier, classifying email, and checking validation error

- Training the perceptron liner classifier with training set
- Ensuring training error is 0
- Classifying email within the validation set
- Checking the validation error

### 5 - Identifying words with highest and lowest weight

- In order to understand which the workings of the spam classifier, step 5 attempts to identify:

    a) 15 words with most positive weights 
    
    b) 15 words with least positive (most negative) weights
    
### 6 - Averaged Perceptron Algorithm 

- Returning the average of all weight vectors considered during 

### 7 - Preventing Overfitting: Limiting maximum number of passes over data

- Implement a hyperparameter that controls the number of passes over data as for large datasets the algorithm may take in numerous iterations for minimal change, causing overfitting

### 8 - Experimenting with varying value of minimum word appearance level

### 9 - Final Test: Learning with best configuration and testing on test dataset



### Part 1
This problem set will involve your implementing several variants of the Perceptron algorithm. Before you can build these models and measure their performance, split your
training data (i.e. spam train.txt) into a training and validate set, putting the last 1000
emails into the validation set. Thus, you will have a new training set with 4000 emails and
a validation set with 1000 emails. You will not use spam test.txt unti

In [36]:
import numpy as np

In [37]:
# Part 1: Defining a function that divides the dataset

def divideDataSet(inputFile): 
    
    # Inital arrays for training dataset and validation dataset defined
    trainingData = [] 
    validationData = []
    
    increment = 1
    
    # Looping through the input (which is 5000 lines) to divide the dataset into 4000 training dataset and 1000 validation dataset
    for i in inputFile:
        if increment > 4000:
            validationData.append(i)
        else:
            trainingData.append(i)
        increment += 1
        
    inputFile.close()
    
    # Modidying the data type from an array to a string for preparation
    validationData = " ".join(validationData)
    trainingData = " ".join(trainingData)
    
    # Creating separate text files for validation dataset and training dataset
    validationFile = open("spam_validation.txt", "w")
    trainingFile = open("spam_train.txt", "w")

    # Writing
    trainingFile.write(trainingData) 
    validationFile.write(validationData)    

In [38]:
# Running the divider function

dataset = open("Data/spam_train.txt", "r")
divideDataSet(dataset)

#### Response to Question: Problem with measuring performance without validation set
Measuring the performance of the final classifier would be problematic without the validation set because the only way to test the validity and performance of the model would be to use the test dataset, which is finite and is reserved strictly for testing. Test dataset should only be used for <b>testing</b> since exposing our model to the test dataset will optimize our model based on the real data that should have been used for the final performance test, creating an unfavorable scenario.

### Part 3

Transform all of the data into feature vectors. Build a vocabulary list using only the 4000 e-mail training set by finding all words that occur across the training set. Note that we assume that the data in the validation and test sets is completely unseen when we train our model, and thus we do not use any information contained in them. Ignore all words that appear in fewer than X = 30 e-mails of the 4000 e-mail training set – this is both a means of preventing overfitting and of improving scalability. For each email, transform it into a feature vector x where the ith entry, xi, is 1 if the ith word in the vocabulary occurs in the email, and 0 otherwise.

In [39]:
# Part 2: Vectorizing the emails

def emailVectorizer(inputData):
    
    # Dictionary of vocabularies and their frequency of appearance
    vocabFrequencyDict = {} 
    
    # Array composed of the emails vectorized in terms of their constituent words
    vectorList = [] 

    # List identifying whether email is spam or ham
    spamIdentify = [] 
    
    # inputData is converted into an array that is split by newline
    dataList = inputData.split("\n")
    
    #looping through the number of dataset inside the dataList array (4000)
    for i in range(len(dataList)):
        # Dictionary for vocabulary and its frequency of appearance within email 'i'
        vocabOccurence = {}
        
        # An array of inidividual words within email i
        vocabulary = dataList[i].split()
        
        # *** The for loop starts form index 1 as index 0 is a number 0/1 that identifies spam or ham
        for vocab in vocabulary [1:]:
            if vocab not in vocabOccurence:
                if vocab not in vocabFrequencyDict:
                    vocabFrequencyDict.update({vocab: 1})
                else:
                    vocabFrequencyDict[vocab] += 1
                vocabOccurence.update({vocab: 1})
    
        # As aforementioned, since the first index of vocabulary is 0/1, identifying whether the email is spam or ham
        if len(vocabulary) > 0:
            vectorList.append(vocabOccurence) 
            spamIdentify.append(int(vocabulary[0]))
            
    return vocabFrequencyDict , vectorList , spamIdentify




#### emailVectorizer(inputData) is a function that vectorizes the email by creating:
1. vocabFrequencyDict - a dictionary that contains vocabulary and its frequency of appearance in emails
2. vectorList - a list of dictionaries that contain the vocabulary within individual email
3. spamIdentify - a list that identifies whether email is spam or ham 



In [40]:
# Part 2: Building the vocabulary list 

def makeWordList(vocabFrequencyDict , minEmail):
    # An Array of words in emails and Dictionary of words mapped with numerical index
    wordList = [] 
    wordMap = {} 
    
    inc = 0
    
    # Looping through the Frequency Dictionary in order to filter out words appearing in less than 'minemail' emails
    for key in vocabFrequencyDict.keys(): 
        # minEmail value can be changed by the user as needs be
        if vocabFrequencyDict[key] >= minEmail:
            wordList.append(key) 
            # Creaitng this dictionary of words allow us to easily identify and access 
            wordMap.update({key: inc})
            inc += 1
            
    return wordMap , wordList


#### makeWordList((vocabFrequencyDict , minEmail) creates a vocabulary list of words that appear in more than 'minemail' emails

1. wordMap - a dictionary that maps the vocabulary to index
2. wordList - a list of vocabulary that appears in more than 'minEmail' emails

In [41]:
# Part 2: Building the feature list using the vocab list

def makeFeatureList(wordMap, emailVector):
    # 2D Array that contains features
    featureList = []
    for emailDict in emailVector:
        features = [0 for i in range(len(wordMap.keys()))]
        for key in emailDict.keys():
            if key in wordMap:
                features[wordMap[key]] = 1
        featureList.append(features)
    return np.array(featureList)


#### makeFeatureList(wordMap, emailVector) creates a feature list that indicates whether a specific vocabulary exists in the wordMap. 
- Numpy is used in order to increase performance (i.e. dot products) 

Source: https://becominghuman.ai/an-essential-guide-to-numpy-for-machine-learning-in-python-5615e1758301

### Part 3

Implement the functions perceptron train(data) and perceptron test(w, data).
The function perceptron train(data) trains a perceptron classifier using the examples
provided to the function, and should return w, k, and iter, the final classification vector,
the number of updates (mistakes) performed, and the number of passes through the data,
respectively. You may assume that the input data provided to your function is linearly
separable (so the stopping criterion should be that all points are correctly classified). For
the corner case of w · x = 0, predict the +1 (spam) class.

The function perceptron test(w, data) should take as input the weight vector w (the
classification vector to be used) and a set of examples. The function should return the test
error, i.e. the fraction of examples that are misclassified by w.

In [122]:

def trainPerceptron(dataInput):
    # Retrieving vectorized emails and the feature list
    a , emailVector , spamList = emailVectorizer(dataInput)
    featureList = makeFeatureList(wordMap, emailVector)

    weight = []
    for i in range(len((featureList[0]))):
        weight.append(0)

    # Epochs: the number of repeated iterations    
    repeatCount = 0 

    # Total Error on updates
    totalError = 0 

    # Current iteration's update num
    curIter = -1 


    while not (curIter == 0):
        curIter = 0 
        count = 0
        while count < len(featureList):
            if spamList[count] == 0:
                checker =-1
            else: 
                checker=1
            # Dot product is calculated by multiplying feature and weight
            product = np.dot(featureList[count], weight) 
            if product == 0:
                product = 1
            # Use checker to identify sign
            if product*checker <= 0:
                curIter += 1 
                weight += checker*(featureList[count])
            count += 1
        # Total Error incremented with current iteration's update number
        totalError += curIter
        repeatCount += 1
    
    return weight , totalError , repeatCount

def testPerceptron(weight , dataInput): 
    # Getting the featureList
    a , emailVector , spamList = emailVectorizer(dataInput)
    featureList = makeFeatureList(wordMap , emailVector)
    noErrors = 0 
    count = 0
    while count < len(featureList):
    # Get the checker we use to multiple with dot product
        if spamList[count] == 0: 
            checker =-1
        else: 
            checker=1
        # Dot product is calculated by multiplying feature and weight
        product = np.dot(featureList[count], weight) 
        if product == 0:
            product = 1
        # Use checker to identify sign
        if product*checker <= 0:
            noErrors += 1
        
        count += 1
    return float(noErrors/count)
                
                
                
                

In [123]:
def readData(trainInput, validationInput):
    trainData = open(trainInput).read()
    validationData = open(validationInput).read() 
    
    return trainData, validationData

### Part 4
Train the linear classifier using your training set. How many mistakes are made before the
algorithm terminates? Test your implementation of perceptron test by running it with
the learned parameters and the training data, making sure that the training error is zero.
Next, classify the emails in your validation set. What is the validation error?

In [124]:
# Part 4

# Retrieving Datasets
trainData, validationData = readData("spam_train.txt", "spam_validation.txt")

# Vectorizing email and creating vocab list
vocabFrequencyDict , emailVector , a = emailVectorizer(trainData)
wordMap, wordList = makeWordList(vocabFrequencyDict,30)
    
# Training the perceptron classifier
weight, totalError, repeatCount = trainPerceptron(trainData)
testError = testPerceptron(weight , trainData) 
testErrorValidate = testPerceptron(weight , validationData)

# Priting the results
print("Training Set Error Level: ", testError)
print("Number of mistakes made by trainPerceptron:", totalError) 
print("Validation Set Error Level: ", testErrorValidate)

This is the error on the training set:  0.0
Number of mistakes made by trainPerceptron: 447
This is the error on the validation set:  0.02


### Part 5
To better understand how the spam classifier works, we can inspect the parameters to see
which words the classifier thinks are the most predictive of spam. Using the vocabulary list together with the parameters learned in the previous question, output the 15 words
with the most positive weights. What are they? Which 15 words have the most negative
weights?

In [135]:

def spamHamIdentify(weight, limit):
    # Mapping weight to vocab
    sortVocab = {vocab:weight for (vocab, weight) in zip(wordList, weight)}
    
    # Sorting dictionary using sorted()
    sortVocab=sorted(sortVocab.items(), key=lambda x: x[1])
    
    spamList=sortVocab[:limit] 
    hamList=(list(reversed(sortVocab[-limit:]))) 
    
    return (spamList,hamList)    


In [136]:
# Retrieving Datasets
trainData, validationData = readData("spam_train.txt", "spam_validation.txt")

# Vectorizing email and creating vocab list
vocabFrequencyDict , emailVector , a = emailVectorizer(trainData)
wordMap, wordList = makeWordList(vocabFrequencyDict,30)

# Training the perceptron classifier
weight, totalError, repeatCount = trainPerceptron(trainData)

# Calling the spam or ham identification function
spamList,hamList=spamHamIdentify(weight,15)

#Printing Results
print("15 most positive vocabulary and weight in order = ", hamList)
print("15 most negative vocabulary and weight in order= ", spamList)


15 most positive vocabulary and weight in order =  [('sight', 19), ('our', 17), ('yourself', 16), ('remov', 16), ('guarante', 15), ('market', 15), ('nbsp', 15), ('pleas', 15), ('click', 15), ('these', 15), ('deathtospamdeathtospamdeathtospam', 14), ('ever', 14), ('present', 14), ('your', 14), ('major', 13)]
15 most negative vocabulary and weight in order=  [('but', -15), ('wrote', -15), ('prefer', -15), ('and', -14), ('i', -13), ('reserv', -13), ('on', -12), ('still', -12), ('technolog', -12), ('sinc', -11), ('copyright', -11), ('url', -11), ('instead', -11), ('upgrad', -11), ('recipi', -11)]


### Part 6
Implement the averaged perceptron algorithm, which is the same as your current implementation but which, rather than returning the final weight vector, returns the average
of all weight vectors considered during the algorithm (including examples where no mistake was made). Averaging reduces the variance between the different vectors, 

In [105]:

def trainPerceptronAvg(dataInput):
    # Retrieving vectorized emails and the feature list
    a , emailVector , spamList = emailVectorizer(dataInput)
    featureList = makeFeatureList(wordMap, emailVector)

    weight = []
    for i in range(len((featureList[0]))):
        weight.append(0)
    
    # numpy array for operational modifications
    weightAvg = np.array([0 for i in range(len(featureList [0]))])
    
    # Epochs: the number of repeated iterations
    repeatCount = 0
    
    # Current iteration's update num
    curIter = -1 

    while not (curIter == 0): 
        curIter = 0
        count = 0
        while count < len(featureList):
            # Get the y we use to multiple with dot product
            if spamList[count] == 0:
                checker = -1 
            else:
                checker = 1
            # Dot product is calculated by multiplying feature and weight
            product = np.dot(featureList[count], weight)
            if product == 0:
                product = 1
            # Use checker to identify sign
            if product*checker <= 0:
                curIter += 1 
                weight += checker*(featureList[count])
            count += 1
            avgRepeats = (repeatCount+1)*count
            weightAvg = (weightAvg*(avgRepeats -1) + weight)/ avgRepeats

        repeatCount += 1
    return weightAvg


### Part 7
Add an argument to both the perceptron and the averaged perceptron that controls the maximum
number of passes over the data. This is an important hyperparameter because for large training
sets, the perceptron algorithm can take many iterations just changing a small subset of the point -- leading to overfitting.

In [106]:
# Part 7

def trainPerceptron(dataInput, noPass):
    # Global variables for outside function modification
    global wordMap 
    global wordList 
    
    # Retrieving vectorized emails and the feature list
    vocabFrequencyDict , emailVector , spamList = emailVectorizer(dataInput)
    wordMap, wordList = makeWordList(vocabFrequencyDict, 30)
    featureList = makeFeatureList(wordMap , emailVector)

    weight = []
    for i in range(len((featureList[0]))):
        weight.append(0)

    # Epochs: the number of repeated iterations
    repeatCount = 0 

    # Total Error on updates
    totalError = 0 

    # Current iteration's update num
    curIter = -1 
    
    
    while not (curIter == 0) and ((noPass == -1) or repeatCount < noPass):
        curIter = 0 
        count = 0

        while count < len(featureList):
            if spamList[count] == 0: 
                checker = -1
            else: 
                checker = 1
            # Dot product is calculated by multiplying feature and weight
            product = np.dot(featureList[count], weight) 
            if product == 0:
                product = 1
            # Use checker to identify sign
            if product*checker <= 0:
                curIter += 1 
                weight += checker*(featureList[count])
            count += 1
        # Total Error incremented with current iteration's update number
        totalError += curIter
        repeatCount += 1
        
    return weight , totalError , repeatCount

def trainPerceptronAvg(dataInput, noPass):

    # Global variables for outside function modification
    global wordMap 
    global wordList
    
    # Retrieving vectorized emails and the feature list
    wordMap , emailVector , spamList = emailVectorizer(dataInput)
    wordMap, wordList = makeWordList(vocabFrequencyDict, 30)
    featureList = makeFeatureList(wordMap , emailVector)
    
    weight = []
    for i in range(len((featureList[0]))):
        weight.append(0)
    
    # numpy array for operational modifications
    weightAvg = np.array([0 for i in range(len(featureList [0]))])
    
    # Epochs: the number of repeated iterations
    repeatCount = 0 
    
    # Current iteration's update num
    curIter = -1 

    while not (curIter == 0) and ((noPass == -1) or repeatCount < noPass): 
        curIter = 0
        count = 0
        while count < len(featureList):
            if spamList[count] == 0:
                checker =-1 
            else:
                checker=1
                
            # Dot product is calculated by multiplying feature and weight
            product = np.dot(featureList[count], weight)
            if product == 0: 
                product = 1

            # Use checker to identify sign
            if product*checker <= 0:
                curIter += 1
                weight += checker*(featureList[count])
            
            count += 1
            avgRepeats = (repeatCount+1)*count
            # Calculating the average weight by repeatedly calulating average from the previous average value
            weightAvg = (weightAvg*(avgRepeats -1) + weight) / avgRepeats
        repeatCount += 1
    return weightAvg
    

The error for the training set convergedto 0.018 after 10 iterations. Hence hyperparamter was set accordingly.

### Part 8
Experiment with various maximum iterations on the two algorithms checking performance on the validation set. Optionally you can try to change X from question 2. Report the best validation error for the two algorithms.

In [113]:
# Part 8

trainData, validationData = readData("spam_train.txt", "spam_validation.txt")

vocabFrequencyDict , emailVector , a = emailVectorizer(trainData)
wordMap, wordList = makeWordList(vocabFrequencyDict,30)

# based on testing, the error value converges to 0.018 after 10 iterations
for i in range(10):
    weight, totalError, repeatCount = trainPerceptron(trainData , i+1) # max 11
    testErrorValidate = testPerceptron(weight , validationData)
    weightAvg = trainPerceptronAvg(trainData , i+1)
    testErrorValidateAverage = testPerceptron(weightAvg , validationData)

    print("#",i+1,"iteration - trainPerceptron:", testErrorValidate)
    print("#",i+1,"iteration - trainPerceptronAvg:",testErrorValidateAverage)

# 1 iteration - trainPerceptron: 0.028
# 1 iteration - trainPerceptronAvg: 0.02
# 2 iteration - trainPerceptron: 0.029
# 2 iteration - trainPerceptronAvg: 0.02
# 3 iteration - trainPerceptron: 0.047
# 3 iteration - trainPerceptronAvg: 0.018
# 4 iteration - trainPerceptron: 0.018
# 4 iteration - trainPerceptronAvg: 0.019
# 5 iteration - trainPerceptron: 0.027
# 5 iteration - trainPerceptronAvg: 0.015
# 6 iteration - trainPerceptron: 0.024
# 6 iteration - trainPerceptronAvg: 0.02
# 7 iteration - trainPerceptron: 0.02
# 7 iteration - trainPerceptronAvg: 0.021
# 8 iteration - trainPerceptron: 0.025
# 8 iteration - trainPerceptronAvg: 0.02
# 9 iteration - trainPerceptron: 0.02
# 9 iteration - trainPerceptronAvg: 0.019
# 10 iteration - trainPerceptron: 0.02
# 10 iteration - trainPerceptronAvg: 0.018


### Part 9
Combine the training set and the validation set (i.e. us all of spam_train.txt) and learn using the best of the configurations previously found. You do not need to rebuild the vocabulary when re-training on the train + validate set. What is the error on the test set (i.e. spam_test.txt).

In [114]:
# Part 9

trainData, validationData = readData("spam_train.txt", "spam_validation.txt")
testData = open("Data/spam_test.txt").read()

vocabFrequencyDict , emailVector , a = emailVectorizer(trainData)
wordMap, wordList = makeWordList(vocabFrequencyDict,30)

#Final Testing with test dataset
finalWeight = trainPerceptronAvg(validationData, 5) 
finalError = testPerceptron(finalWeight , testData)
print("Error on Test Set:", finalError)

Error on Test Set: 0.042
